In [28]:
from bs4 import BeautifulSoup
import pandas as pd
import random
import time
import requests

In [29]:
def fetch_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    proxy_list = []

    table_rows = soup.find('table' , class_='table table-striped table-bordered').find('tbody').find_all('tr')
    
    for row in table_rows:
        cols = row.find_all("td")
        ip = cols[0].text
        port = cols[1].text
        https = cols[6].text
        if https == "yes":
            proxy_list.append(f"{ip}:{port}")

    return proxy_list

In [30]:
def validate_proxies(proxies, test_url="https://httpbin.org/ip", timeout=5):
    working = []
    for proxy in proxies:
        try:
            res = requests.get(test_url, proxies={
                "http": f"http://{proxy}",
                "https": f"http://{proxy}"
            }, timeout=timeout)
            if res.status_code == 200:
                print(f"[✅] Working: {proxy}")
                working.append(proxy)
        except:
            print(f"[❌] Failed: {proxy}")
        time.sleep(1)
    return working

In [31]:
def scrape_with_rotation(url, proxy_list):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }

    for attempt in range(10):
        proxy = random.choice(proxy_list)
        print(f"Trying with proxy: {proxy}")
        try:
            res = requests.get(url, proxies={
                "http": f"http://{proxy}",
                "https": f"http://{proxy}"
            }, headers=headers, timeout=10)
            if res.status_code == 200:
                print(f"[✅] Success with {proxy}")
                return res.text
        except Exception as e:
            print(f"[⚠️] Failed with {proxy}: {e}")
    print("❌ All proxies failed.")
    return None

In [32]:
titles = []
prices = []
links  = []

In [44]:
def aliexpress_page_scraper(html):

    soup = BeautifulSoup(html , 'html.parser')
    products = soup.find_all('div' , class_ = 'kr_k3')

    for product in products:

        product_title = product.find('div', class_ = 'kr_av').get('title')
        product_price = product.find('div', class_ = 'kr_kj').text
        product_link  = product.find('a', class_ = 'kr_b in_is search-card-item').get('href')

        titles.append(product_title)
        prices.append(product_price)
        links.append(product_link)

In [34]:
all_proxies = fetch_proxies()
proxy_list = validate_proxies(all_proxies[:50])

[❌] Failed: 51.81.245.3:17981
[❌] Failed: 47.236.224.32:8080
[✅] Working: 45.12.150.82:8080
[❌] Failed: 139.59.34.209:8080
[❌] Failed: 66.201.7.151:3128
[❌] Failed: 37.27.63.151:29923
[❌] Failed: 57.129.81.201:8080
[❌] Failed: 72.10.160.174:3295
[✅] Working: 77.90.28.184:1080
[❌] Failed: 31.40.248.2:8080
[❌] Failed: 103.3.58.162:3128
[❌] Failed: 71.14.218.2:8080
[✅] Working: 159.69.57.20:8880
[❌] Failed: 8.219.97.248:80
[❌] Failed: 200.174.198.86:8888
[✅] Working: 185.234.65.66:1080
[❌] Failed: 188.245.239.104:4001
[❌] Failed: 68.183.63.141:8080
[❌] Failed: 45.22.209.157:8888
[❌] Failed: 197.44.247.35:3128


In [46]:
total_pages = 5

for page in range(1,total_pages+1):

    url = f'https://www.aliexpress.com/w/wholesale-airpods.html?page={page}'
    html = scrape_with_rotation(url , proxy_list)

    aliexpress_page_scraper(html)

Trying with proxy: 45.12.150.82:8080
[✅] Success with 45.12.150.82:8080
Trying with proxy: 185.234.65.66:1080
[✅] Success with 185.234.65.66:1080
Trying with proxy: 45.12.150.82:8080
[✅] Success with 45.12.150.82:8080
Trying with proxy: 45.12.150.82:8080
[✅] Success with 45.12.150.82:8080
Trying with proxy: 185.234.65.66:1080
[✅] Success with 185.234.65.66:1080


In [49]:
url = 'https://www.aliexpress.com/w/wholesale-gaming-keyboards.html'
html = scrape_with_rotation(url , proxy_list)
with open('page_1.html' , 'w' , encoding='utf-8') as file:
    file.write(html)
    
aliexpress_page_scraper(html)

Trying with proxy: 77.90.28.184:1080
[✅] Success with 77.90.28.184:1080


In [55]:
soup = BeautifulSoup(html , 'html.parser')
products = soup.find_all('div' , class_ = 'kr_k3')

for product in products:

    product_title = product.find('div', class_ = 'kr_av').get('title')
    print(product_title)
    product_price = product.find('div', class_ = 'kr_kj').text
    product_link  = product.find('a', class_ = 'kr_b in_is search-card-item').get('href')

    titles.append(product_title)
    prices.append(product_price)
    links.append(product_link)

In [53]:
prices

[]

In [38]:
df = pd.DataFrame({
    'Titles' : titles,
    'Prices' : prices,
    'Links'  : links
})

df.shape

ValueError: All arrays must be of the same length

In [ ]:
df = df[df['Links'].str.contains('item')]
df.reset_index( drop=True , inplace=True)
df.index = range(1,len(df)+1)
df.shape

In [ ]:
df.to_csv('aliexpress_products.csv')
df.head()